In [0]:
! pip install sigopt

In [0]:
# Run `pip install sigopt` to download our python API client
from sigopt import Connection
from sigopt.examples import franke_function

# define our model to be the Franke Function
def evaluate_model(assignments):
    return franke_function(assignments['x'], assignments['y'])
  
# put your api_token here
api_token = "XXXXXXXXXXXX"
  
conn = Connection(client_token=api_token)

In [0]:
# Create a SigOpt experiment
experiment = conn.experiments().create(
    name='Seeding SigOpt with Grid Search',
    project='sigopt-examples',
    metrics=[dict(name='function_value', objective='maximize')],
    parameters=[
        dict(name='x', type='double', bounds=dict(min=0.0, max=1.0)),
        dict(name='y', type='double', bounds=dict(min=0.0, max=1.0)),
    ],
)
print("Explore your experiment: https://app.sigopt.com/experiment/" + experiment.id + "/analysis")

# Start with grid search and use the output to seed SigOpt
Seed SigOpt with a user-defined GridSearch

In [0]:
# Specifiy 25 grid search points
xs = [0.0, 0.25, 0.5, 0.75, 1.0]
ys = [0.0, 0.25, 0.5, 0.75, 1.0]

for x in xs:
    for y in ys:
        # evaluate next grid search point
        value = evaluate_model({"x": x, "y": y})

        # pass grid search observation to SigOpt
        observation =  conn.experiments(experiment.id).observations().create(
          assignments = {"x": x, "y": y},
          value = value,
          no_optimize = True
        )
    
# Use SigOpt to generate 25 more intelligent suggestions using info from grid to bootstrap
for _ in range(25):
    suggestion = conn.experiments(experiment.id).suggestions().create()
    value = evaluate_model(suggestion.assignments)
    conn.experiments(experiment.id).observations().create(
        suggestion = suggestion.id,
        value = value,
    )
    
print("Explore your experiment: https://app.sigopt.com/experiment/" + experiment.id + "/analysis")

# Import a Pandas DataFrame into SigOpt to seed an experiment with > 2 metrics

Here we generate a 6 column Pandas DataFrame that represents a 2 dimensional problem (since we are evaluating the 2D franke function) with 4 output metric values. Below is example code that can be used to seed a SigOpt experiment with those 100 observations and continue intelligently searching the space from there.

In [0]:
# create pandas dataframe
import numpy as np
import pandas as pd
random_df = pd.DataFrame(np.random.rand(100, 6), columns=list('ABCDEF'))


In [0]:
# Create a SigOpt experiment
experiment = conn.experiments().create(
    name='Seeding SigOpt with a Dataframe',
    project='sigopt-examples',
    metrics=[dict(name='function_value', objective='maximize')],
    parameters=[
        dict(name='x', type='double', bounds=dict(min=0.0, max=1.0)),
        dict(name='y', type='double', bounds=dict(min=0.0, max=1.0)),
    ],
    metadata = dict(metric2=-10000.0, metric3=-10001.0, metric4=-10002.0)
)
print("Explore your experiment: https://app.sigopt.com/experiment/" + experiment.id + "/analysis")

In [0]:
# define parameter names 
suggestion_key_list = ["x","y"]
unoptimized_metric_key_list = ["C", "D", "E"]

for row in random_df.iterrows():
  
  # get row of pandas data frame corresponding to suggestion
  suggestion_as_list = row[1].values
  # in this example 1 suggestion (1 row of the df) contains 
    # 2 parameters (columns A and B)
    # 4 metrics (columns C, D, E, and F)
      # Say that column F is the metric we care to optimize 
      # but other 3 metrics are important to track as well
  
  param_assignments = {}
  for i, key in enumerate(suggestion_key_list):
    param_assignments[key] = suggestion_as_list[i]
    
  background_metrics = {}
  for i in range(len(suggestion_key_list), len(suggestion_as_list) - 1):
    background_metrics[unoptimized_metric_key_list[i - len(suggestion_key_list)]] = suggestion_as_list[i]

  # for this example, the optimized metric is stored in the last column of the dataframe
  optimized_metric = suggestion_as_list[-1]
  
  # pass loaded observations to SigOpt
  observation =  conn.experiments(experiment.id).observations().create(
    assignments = param_assignments,
    value = optimized_metric,
    metadata = background_metrics,
    no_optimize = True
  )
  

In [0]:
# evaluate metadata on parameters assignments['x'], assignments['y']
# these will be tracked alongside the optimized metric 

def get_metadata(assignments):
  return {"C": np.random.rand(), "D": np.random.rand(), "E": np.random.rand()}

In [0]:
# Use SigOpt to generate 100 new intelligent suggestions
for _ in range(100):
  
    suggestion = conn.experiments(experiment.id).suggestions().create()
    
    param_assignments  = suggestion.assignments
    optimized_metric   = evaluate_model(param_assignments) 
    background_metrics = get_metadata(param_assignments)
    
    conn.experiments(experiment.id).observations().create(
        suggestion = suggestion.id,
        value      = value,
        metadata   = background_metrics,
    )

#Export SigOpt observations to Pandas Dataframe

Here we will fetch all SigOpt observations through the API and store them in a Pandas Dataframe. 

In [0]:
observations = conn.experiments(experiment.id).observations().fetch()

In [0]:
df = pd.DataFrame(columns=list('ABCDEF'))

for i, obs in enumerate(observations.data):
  
  param_assignments = obs.assignments
  background_metrics = obs.metadata
  
  A = param_assignments['x']
  B = param_assignments['y']
  C = background_metrics['C']
  D = background_metrics['D']
  E = background_metrics['E']
  F = obs.value # optimized metric value
  
  df.loc[i] = [A, B, C, D, E, F]
  